This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [7]:
structure_name = "LLZO_010_Li_order0_off__Li_111_slab_heavy"

In [8]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_Li_order0_off__Li_111_slab_heavy
- Initial lattice height: 69.41 Å
- Number of atoms: 908


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.26 Å
Lowest Li slab atom: 33.16 Å
LLZO top z: 32.79 Å
Li penetration into LLZO: -0.36 Å


In [10]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 17.39 Å
Li slab thickness:   21.26 Å
LLZO top z:          32.79 Å
Lowest Li atom z:    33.16 Å
Li penetration into LLZO: -0.36 Å


In [11]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 654 atoms out of 908
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_170512/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [12]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 18:36:17    -2280.032521        3.880850
SciPyFminCG:    1 18:36:20    -2296.733185        2.831892
SciPyFminCG:    2 18:36:25    -2319.748905        2.166802
SciPyFminCG:    3 18:36:29    -2342.489258        1.660980
SciPyFminCG:    4 18:36:32    -2347.773417        1.596737
SciPyFminCG:    5 18:36:35    -2349.613316        1.225994
SciPyFminCG:    6 18:36:38    -2350.490076        0.940177
SciPyFminCG:    7 18:36:41    -2351.154249        0.946846
SciPyFminCG:    8 18:36:44    -2352.219567        1.202193
SciPyFminCG:    9 18:36:47    -2352.967953        1.072792
SciPyFminCG:   10 18:36:50    -2353.435991        1.020584
SciPyFminCG:   11 18:36:53    -2353.976768        0.907405
SciPyFminCG:   12 18:36:56    -2354.513649        1.400464
SciPyFminCG:   13 18:36:59    -2355.250344        1.459820
SciPyFminCG:   14 18:37:00    -2355.531990        1.431521
SciPyFminCG:   15 18:37:04    -2356.042027        0.549908

In [13]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_170512/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 18:45:52    -2405.409487        0.128475
FIRE:    1 18:45:54    -2405.412301        0.065472
FIRE:    2 18:45:55    -2405.413383        0.087492
FIRE:    3 18:45:57    -2405.414249        0.094519
FIRE:    4 18:45:59    -2405.416847        0.055142
FIRE:    5 18:46:00    -2405.418363        0.079551
FIRE:    6 18:46:01    -2405.420311        0.085808
FIRE:    7 18:46:03    -2405.421610        0.077980
FIRE:    8 18:46:04    -2405.424424        0.086041
FIRE:    9 18:46:06    -2405.427671        0.100824
FIRE:   10 18:46:07    -2405.431135        0.079473
FIRE:   11 18:46:09    -2405.434599        0.107674
FIRE:   12 18:46:10    -2405.439145        0.123960
FIRE:   13 18:46:12    -2405.444124        0.151521
FIRE:   14 18:46:13    -2405.451918        0.292140
FIRE:   15 18:46:15    -2405.459711        0.427988
FIRE:   16 18:46:16    -2405.462742        0.205227
FIRE:   17 18:46:18    -2405.464041        0.400437
FIRE:   18 18:

In [14]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [15]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.64 Å
Lowest Li slab atom: 32.77 Å
LLZO top z: 32.64 Å
Li penetration into LLZO: -0.13 Å


In [16]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 18:47:48    -2405.561892        3.869458
SciPyFminCG:    1 18:47:52    -2430.020695        4.215679
SciPyFminCG:    2 18:47:56    -2456.280429        4.543577
SciPyFminCG:    3 18:47:58    -2482.924206        3.021827
SciPyFminCG:    4 18:48:01    -2506.647396        4.417941
SciPyFminCG:    5 18:48:04    -2526.659833        3.326260
SciPyFminCG:    6 18:48:07    -2537.230101        1.899958
SciPyFminCG:    7 18:48:09    -2543.300314        1.900111
SciPyFminCG:    8 18:48:12    -2547.553576        2.186227
SciPyFminCG:    9 18:48:15    -2554.351826        2.059545
SciPyFminCG:   10 18:48:17    -2560.996371        1.905422
SciPyFminCG:   11 18:48:20    -2564.314423        2.036374
SciPyFminCG:   12 18:48:23    -2568.544738        1.471270
SciPyFminCG:   13 18:48:26    -2571.809318        1.632039
SciPyFminCG:   14 18:48:28    -2574.877113        1.487657
SciPyFminCG:   15 18:48:31    -2577.903561        1.234107

In [17]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_170512/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 19:03:29    -2795.980304        0.590621
FIRE:    1 19:03:30    -2796.010395        0.403641


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.953234722606529e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:    2 19:03:31    -2796.038538        0.207777


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.9841894874393533e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


...

FIRE:  398 19:10:16    -2797.402821        0.124679


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.939874822526541e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  399 19:10:17    -2797.412562        0.180670


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.0086343945285807e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  400 19:10:18    -2797.420139        0.502141


In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_Li_order0_off__Li_111_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 